## Import

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from metric import recall5, recall1

C:\Users\User\anaconda3\envs\git_repo\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Data Load

In [3]:
apply_train_df = pd.read_csv('apply_train.csv').sort_values('resume_seq')

def sample_group(group, n=1):
    return group.sample(n)

# groupby를 사용하여 그룹화하고 샘플링 함수를 적용
test = apply_train_df.groupby('resume_seq', group_keys=False).apply(sample_group, n=2).reset_index(drop=True)
train = apply_train_df.drop(test.index).reset_index(drop=True)



# # 그룹화한 후 그룹 크기 계산
# grouped = apply_train_df.groupby('resume_seq')
# group_sizes = grouped.size()

# # 그룹 크기가 5개 이상인 그룹 선택
# selected_groups = group_sizes[group_sizes >= 10].index

# # 선택한 그룹만 필터링
# apply_train_df = apply_train_df[apply_train_df['resume_seq'].isin(selected_groups)]

# apply_train_df

## User-Item Matrix / Similarity / Score

In [4]:
# 사용자-아이템 행렬 생성: 구직자가 해당 채용 공고에 지원했으면 1, 아니면 0으로 설정
user_item_matrix = apply_train_df.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
user_item_matrix[user_item_matrix > 1] = 1

# 사용자 간의 유사성 계산
user_similarity = cosine_similarity(user_item_matrix)

# 추천 점수 계산
user_predicted_scores = user_similarity.dot(user_item_matrix) / np.array([np.abs(user_similarity).sum(axis=1)]).T

In [5]:
user_item_matrix

recruitment_seq,R00001,R00002,R00003,R00004,R00005,R00006,R00007,R00008,R00009,R00010,...,R06686,R06687,R06688,R06689,R06690,R06691,R06692,R06693,R06694,R06695
resume_seq,,,,,,,,,,,,,,,,,,,,,
U00001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U00002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U00003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U00004,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U00005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
U08478,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U08479,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U08480,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
user_similarity.dot(user_item_matrix).shape

(8482, 6695)

In [65]:
tp = pd.read_csv('./recruitment.csv')
req_list = [list(i['recruitment_seq']) for i in pd.DataFrame(apply_train_df.groupby('resume_seq'))[1]]
target = 0
ans = pd.DataFrame()
for i in [tp[tp['recruitment_seq'] == i] for i in req_list[target]]:
    ans = pd.concat([ans,i])
    
reverse_tracking = [eval(i[2:].lstrip('0')) for i in ans['recruitment_seq']]
ans

,recruitment_seq,address_seq1,address_seq2,address_seq3,career_end,career_start,check_box_keyword,education,major_task,qualifications,text_keyword
219,R05210,3.0,NaN,NaN,0,0,2507;2707,3,2,2,NaN
4589,R05288,3.0,NaN,NaN,0,0,2507,4,7,1,NaN
4595,R04536,3.0,NaN,NaN,0,0,2507;2707;2799,3,2,2,NaN
6027,R06065,3.0,NaN,NaN,0,0,2507,4,7,1,NaN


In [64]:
reverse_tracking = [eval(i[2:].lstrip('0')) for i in ans['recruitment_seq']]

In [66]:
tp = pd.read_csv('./resume.csv')
resu_list = [list(i['resume_seq']) for i in pd.DataFrame(apply_train_df.groupby('recruitment_seq'))[1]]
ans = pd.DataFrame()

for target in reverse_tracking:
#target = 5210
    target -= 1
    for i in [tp[tp['resume_seq'] == i] for i in resu_list[target]]:
        ans = pd.concat([ans,i])
    
ans


,resume_seq,reg_date,updated_date,degree,graduate_date,hope_salary,last_salary,text_keyword,job_code_seq1,job_code_seq2,job_code_seq3,career_month,career_job_code
1690,U00001,2019-12-27,2020-02-01,4,2009,1500.0,1500.0,디자이너,재료·화학·섬유·의복,NaN,NaN,0,NaN
5700,U00803,2020-01-27,2020-01-28,3,2007,1100.0,0.0,NaN,재료·화학·섬유·의복,NaN,NaN,0,NaN
442,U02861,2020-02-01,2020-02-01,3,2008,1700.0,2500.0,머천다이저;디자이너,재료·화학·섬유·의복,NaN,NaN,21,건설;토목;건축
8029,U05173,2019-01-17,2020-02-06,4,2008,1700.0,0.0,의류디자이너;의상디자이너;디스플레이;머천다이져;MD;VMD,재료·화학·섬유·의복,NaN,NaN,12,섬유;봉제;가방;의류
7551,U05680,2020-02-18,2020-02-19,4,2009,0.0,0.0,일본무역;디자이너,재료·화학·섬유·의복,NaN,NaN,0,NaN
744,U06049,2014-09-01,2020-01-20,4,2001,0.0,2900.0,캐주얼;의류;스포츠웨어,재료·화학·섬유·의복,디자인,NaN,72,섬유;봉제;가방;의류
1690,U00001,2019-12-27,2020-02-01,4,2009,1500.0,1500.0,디자이너,재료·화학·섬유·의복,NaN,NaN,0,NaN
5520,U00035,2019-12-18,2020-03-19,4,0,1500.0,1300.0,디자이너,재료·화학·섬유·의복,NaN,NaN,0,NaN
1229,U00930,2019-11-16,2020-01-28,4,2006,0.0,1900.0,패션디자인;전시기획;인테리어;공공디자인,재료·화학·섬유·의복,NaN,NaN,12,섬유;봉제;가방;의류
7695,U01255,2019-03-25,2020-07-08,4,2007,0.0,2100.0,패션디자인의상디자인;상품기획;여성복;남성복;컬러리스트;일러스트;악세사리,재료·화학·섬유·의복,NaN,NaN,30,섬유;봉제;가방;의류


## Prediction

In [4]:
# 이미 지원한 채용 공고 제외하고 추천
recommendations = []
for idx, user in enumerate(user_item_matrix.index):
    # 해당 사용자가 지원한 채용 공고
    applied_jobs = set(user_item_matrix.loc[user][user_item_matrix.loc[user] == 1].index)
    
    # 해당 사용자의 추천 점수 (높은 점수부터 정렬)
    sorted_job_indices = user_predicted_scores[idx].argsort()[::-1]
    recommended_jobs = [job for job in user_item_matrix.columns[sorted_job_indices] if job not in applied_jobs][:5]
    
    for job in recommended_jobs:
        recommendations.append([user, job])

In [7]:
top_recommendations = pd.DataFrame(recommendations, columns=['resume_seq', 'recruitment_seq'])
recall1(top_recommendations,test)

0.01729243160014213

In [30]:
reco = pd.DataFrame(top_recommendations.groupby('resume_seq'),columns=['resume_seq','set'])
reco['set'] = [list(i['recruitment_seq']) for i in reco['set']]
reco = pd.merge(reco,test,how='left')
sum([1 for i,j in zip(reco['set'],reco['recruitment_seq']) if j in i])/len(reco)

0.01729243160014213

rdata grouping

## Submission

In [5]:
# sample_submission.csv 형태로 DataFrame 생성
top_recommendations = pd.DataFrame(recommendations, columns=['resume_seq', 'recruitment_seq'])

top_recommendations.to_csv('./baseline_submit.csv', index=False)